In [1]:
%cd ..
%matplotlib inline

D:\Projects\Python\PL-Heuristic


In [2]:
import pandas as pd
from scipy import stats 

from analysis.caching import get_cached_results

In [3]:
pd.set_option('display.max_rows', 500)

# Preparation

In [4]:
data = get_cached_results()
data.describe()

,heuristic_classroom_utilisation,heuristic_instruction_size,heuristic_objective,heuristic_percentage_instruction,heuristic_percentage_self_study,heuristic_self_study_size,heuristic_teacher_utilisation,ilp_classroom_utilisation,ilp_instruction_size,ilp_objective,ilp_percentage_instruction,ilp_percentage_self_study,ilp_self_study_size,ilp_teacher_utilisation,experiment
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7199.000000,7199.000000,7199.000000,7199.000000,7199.000000,7199.000000,7199.000000,7200.000000
mean,88.235918,10.668209,6610.209687,59.657344,40.342656,49.523178,66.920573,88.026393,11.666783,6748.484177,65.398423,34.601577,44.583444,67.231820,36.500000
std,11.331711,3.545793,2351.192867,21.667173,21.667173,16.217169,14.763535,11.948423,4.471629,2425.308372,21.486972,21.486972,16.178022,17.161286,20.784048
min,60.240964,5.400000,3469.548383,15.000000,5.625000,15.133333,43.750000,60.240964,5.185185,3540.313588,18.625000,1.125000,5.000000,39.375000,1.000000
25%,78.313253,7.931034,4371.918796,43.421875,19.500000,39.812500,53.750000,78.313253,8.408259,4417.573599,49.625000,15.375000,31.250000,53.750000,18.750000
50%,90.361446,10.279151,6256.421236,61.625000,38.375000,46.263158,58.125000,89.156627,11.428571,7713.877651,65.500000,34.500000,44.000000,62.500000,36.500000
75%,100.000000,12.220543,9011.085109,80.500000,56.578125,63.500000,81.250000,100.000000,14.000000,9152.745958,84.625000,50.375000,55.055556,83.750000,54.250000
max,100.000000,20.985075,9932.599786,94.375000,85.000000,80.000000,100.000000,100.000000,24.360656,10080.377678,98.875000,81.375000,80.000000,100.000000,72.000000


In [5]:
# All performance measure columns. In the data set, these are split by 
# method - one for the ILP, and another for the heuristic.
MEASURES = [
    "objective",
    "instruction_size", "self_study_size",
    "percentage_instruction", "percentage_self_study",
    "classroom_utilisation", "teacher_utilisation"]

EXPERIMENTS = list(range(1, 73))

# Analysis

In [6]:
def results_per_experiment(experiment):
    """
    Computes a dictionary of results per experiment. This is composed of the 
    means of each performance measure, per solution method type (one of ilp,
    heuristic).    
    """
    results = {}
    
    for measure in MEASURES:
        ilp_data = data["ilp_" + measure][data.experiment == experiment]
        heuristic_data = data["heuristic_" + measure][data.experiment == experiment]

        results[measure, 'heuristic'] = heuristic_data.mean()
        results[measure, 'ilp'] = ilp_data.mean()

        results[measure, 'difference'] = (heuristic_data - ilp_data).mean()
        results[measure, 'percentage'] = 100 * results[measure, 'difference'] / results[measure, 'ilp']
        results[measure, 't_stat'] = stats.ttest_1samp(heuristic_data - ilp_data, 0, nan_policy='omit').statistic

    return results

In [7]:
result = pd.DataFrame([results_per_experiment(experiment) for experiment in EXPERIMENTS],
                      columns=[(measure, sub)
                               for measure in MEASURES
                               for sub in ['heuristic', 'ilp', 'difference', 'percentage', 't_stat']])

# This presents only the mean results per experiment
means = result.drop(columns=[column for column in result.columns
                             if column[1] not in ['heuristic', 'ilp']])

# This sets the index starting at 1 (for the experiment numbers), and creates
# a hierarchical column index grouped by performance measure and method
means.index = EXPERIMENTS
means.columns = pd.MultiIndex.from_tuples(means.columns,
                                          names=['Measure', 'Method'])

c:\users\niels\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
c:\users\niels\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
c:\users\niels\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


In [8]:
# This presents the pairwise-differences approach
differences = result.drop(columns=[column for column in result.columns
                             if column[1] not in ['difference', 'percentage', 't_stat']])

differences.index = EXPERIMENTS
differences.columns = pd.MultiIndex.from_tuples(differences.columns,
                                                names=['Measure', 'Statistics'])

## Experiment results

In [9]:
# Used to render this table in the paper
with open('experiment_results.tex', 'w') as file:
    means.to_latex(buf=file, float_format="{:0.2f}".format)

## Means

Mean results per experiment, for each method and performance measure.

In [10]:
means

Measure    objective              instruction_size            self_study_size  \
Method     heuristic          ilp        heuristic        ilp       heuristic   
1        4691.814259  4726.152851        14.648488  15.140250       70.104500   
2        4678.852938  4717.152393        14.484949  15.045750       70.223333   
3        4672.496956  4720.069631        14.513145  15.148205       65.328167   
4        4808.414374  4845.053921        10.765734  12.172777       54.394167   
5        4815.614200  4853.035536        10.795859  12.102173       54.792167   
6        4755.060905  4800.435014        11.829425  12.825381       62.413833   
7        9684.617822  9773.145696        18.227771  22.500860       62.056667   
8        9656.898105  9751.891352        18.100133  22.630385       60.262333   
9        9662.106060  9751.204444        18.168942  22.679756       63.315333   
10       9569.822303  9799.907043        10.558738  13.688373       69.943833   
11       9528.936024  9781.638542        10.455929  13.494287       70.675500   
12       9448.303741  9702.678269        11.319688  14.045540       70.238238   
13       4485.542274  4557.616307        16.589000  16.856500       63.130000   
14       4494.793525  4569.584956        16.512667  16.821250       64.835000   
15       4459.987428  4537.899703        16.580970  16.993590       64.853333   
16       4636.064746  4743.583154        12.101378  12.808470       51.640500   
17       4650.095076  4761.138428        12.120462  12.858750       49.149167   
18       4542.888846  4670.698063        13.481943  14.068922       58.103333   
19       9412.311565  9673.507389        18.743518  22.805181       55.495000   
20       9394.981651  9662.843238        18.684463  22.735732       60.059167   
21       9366.547301  9630.887642        18.776198  22.782453       62.722500   
22       9209.457732  9771.771989        11.047691  13.881681       69.606500   
23       9146.481456  9739.278298        11.204930  13.920237       66.462833   
24       9020.711429  9642.636301        12.323935  14.815586       70.072833   
25       4375.281961  4409.569785         8.238071   9.212214       41.249647   
26       4364.515791  4398.222305         8.151332   9.084809       41.232527   
27       4364.972350  4400.468872         8.113610   9.075181       41.121132   
28       4503.756241  4544.731919         7.060045   7.027167       27.796808   
29       4504.085258  4547.979824         7.076822   7.007333       27.312184   
30       4472.870833  4504.393814         7.266724   7.357059       26.016651   
31       9070.602339  9153.986936        11.933941  12.783895       49.838995   
32       9040.927252  9127.229917        11.921986  12.794201       49.905135   
33       9024.104481  9112.865312        11.913569  12.827113       48.988766   
34       9317.941310  9455.788667         8.531905   8.897880       66.195830   
35       9302.737197  9443.478917         8.630240   8.899749       64.683113   
36       9236.907694  9341.970746         9.516582   9.906772       45.547590   
37       3886.953134  3963.166880        10.746280  11.847413       44.528548   
38       3875.618416  3954.244918        10.608228  11.763525       44.212154   
39       3867.218306  3947.745274        10.560387  11.740023       44.089399   
40       4122.628658  4255.920176         7.965615   7.995821       39.126956   
41       4110.897836  4211.785234         8.492511   8.477167       37.180767   
42       4032.163714  4114.912182         8.985539   9.130392       31.519929   
43       8413.875098  8609.720529        13.838681  14.125000       66.945690   
44       8401.534902  8605.023754        13.789706  14.123250       66.015226   
45       8390.805316  8594.597681        13.720648  14.085250       65.337234   
46       8722.755577  9069.016346         9.826579  10.307880       70.699687   
47       8717.961356  9067.685996         9.895690  10.312101       72.224952   
48       8607.427554  8888.7700

# Differences

Pairwise differences between heuristic and ILP solutions, as means per experiment. A one-sample t-test is performed on the differences, to test the null hypothesis that their population mean is zero.

In [11]:
def format_t_stat(value):
    if pd.isna(value):
        return "0.00"
    
    return "{:0.2f}".format(value)

for measure in MEASURES:
    differences[measure] = differences[measure].apply(
        lambda x: "{0:0.2f}\% ({1})".format(x.percentage, format_t_stat(x.t_stat)),
        axis=1)

differences.columns = differences.columns.droplevel(1)
differences = differences.loc[:, ~differences.columns.duplicated()]

In [12]:
# Used to render this table in the paper
with open('pairwise_differences.tex', 'w') as file:
    differences.to_latex(buf=file, escape=False)

In [13]:
differences

Measure,objective,instruction_size,self_study_size,percentage_instruction,percentage_self_study,classroom_utilisation,teacher_utilisation
1,-0.73\% (-25.56),-3.25\% (-17.46),8.19\% (11.66),-3.43\% (-15.15),10.69\% (15.15),0.00\% (0.00),0.00\% (0.00)
2,-0.81\% (-26.47),-3.73\% (-20.22),6.31\% (6.90),-4.25\% (-17.90),12.89\% (17.90),-0.02\% (-1.00),-0.02\% (-1.00)
3,-1.01\% (-26.48),-4.19\% (-20.09),-6.33\% (-5.38),-6.18\% (-19.99),17.45\% (19.99),0.00\% (0.00),0.00\% (0.00)
4,-0.76\% (-43.83),-11.56\% (-37.44),40.26\% (9.63),-3.02\% (-31.50),23.26\% (31.50),7.99\% (17.84),7.99\% (17.84)
5,-0.77\% (-40.61),-10.79\% (-31.27),58.79\% (11.45),-3.21\% (-27.76),23.69\% (27.76),5.55\% (7.96),5.55\% (7.96)
6,-0.95\% (-40.30),-7.77\% (-32.44),71.81\% (12.49),-4.58\% (-28.88),22.52\% (28.88),0.04\% (0.07),0.04\% (0.07)
7,-0.91\% (-71.69),-18.99\% (-46.58),31.10\% (7.77),-2.68\% (-33.51),31.61\% (33.51),18.62\% (34.43),18.62\% (34.43)
8,-0.97\% (-68.34),-20.02\% (-54.39),21.72\% (5.73),-3.02\% (-38.94),35.74\% (38.94),20.47\% (45.19),20.47\% (45.19)
9,-0.91\% (-71.64),-19.89\% (-52.49),28.34\% (6.69),-2.97\% (-40.97),27.99\% (40.97),18.71\% (25.34),18.71\% (25.34)
10,-2.35\% (-166.27),-22.86\% (-81.81),63.00\% (12.06),-14.21\% (-175.21),289.48\% (175.21),12.65\% (31.72),12.65\% (31.72)


## Percentage differences across all experiments

In [14]:
# Average percentages: note that this works since each experiment has the 
# same number of instances, so we need not weigh the numbers explicitly.
columns = [(measure, 'percentage') for measure in MEASURES]

avg_percentage = result[columns]
avg_percentage.columns = [measure for measure, _ in avg_percentage.columns]

avg_percentage.mean()

objective                 -1.894659
instruction_size          -6.747313
self_study_size           17.234789
percentage_instruction    -9.928788
percentage_self_study     56.238660
classroom_utilisation      0.583940
teacher_utilisation        0.583940
dtype: float64